# Exploratory Data Analysis - Gaia DR3 Stellar Data

This notebook performs initial exploration of the Gaia DR3 dataset to understand the stellar population before applying anomaly detection methods.

## Objectives:
1. Load and examine the processed Gaia DR3 data
2. Create Hertzsprung-Russell diagrams
3. Analyze proper motion distributions
4. Identify known stellar populations
5. Prepare data for anomaly detection

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import our custom modules
import sys
sys.path.append('../src')
from visualization import StellarVisualization

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully!")

## 1. Load the Data

In [ ]:
# Load the processed Gaia DR3 data
data_file = "../data/gaia_dr3_processed.csv"

try:
    df = pd.read_csv(data_file)
    print(f"Loaded {len(df):,} stellar sources")
    print(f"Dataset shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
except FileNotFoundError:
    print(f"Error: {data_file} not found.")
    print("Please run '../src/data_acquisition.py' first to download the data.")

In [ ]:
# Basic data info
if 'df' in locals():
    print("Dataset Info:")
    print(df.info())
    
    print("\nBasic Statistics:")
    print(df.describe())

## 2. Data Quality Assessment

In [ ]:
# Check for missing values
if 'df' in locals():
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        print("Missing values per column:")
        for col, count in missing_values.items():
            if count > 0:
                print(f"  {col}: {count} ({count/len(df)*100:.1f}%)")
    else:
        print("No missing values found!")
    
    # Check data ranges
    print("\nData Ranges:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        print(f"  {col}: [{df[col].min():.3f}, {df[col].max():.3f}]")

## 3. Hertzsprung-Russell Diagram

In [ ]:
# Create HR diagram
if 'df' in locals():
    viz = StellarVisualization()
    
    # Static HR diagram
    fig, ax = viz.plot_hr_diagram(
        df, 
        color_col='bp_rp', 
        mag_col='abs_g_mag',
        title="Gaia DR3 Hertzsprung-Russell Diagram"
    )

In [ ]:
# Interactive HR diagram
if 'df' in locals():
    interactive_fig = viz.create_interactive_hr_diagram(
        df, 
        save_html=True
    )

## 4. Proper Motion Analysis

In [ ]:
# Proper motion analysis
if 'df' in locals():
    fig = viz.plot_proper_motion_distribution(df)

## 5. Feature Correlations

In [ ]:
# Feature correlation analysis
if 'df' in locals():
    fig, corr_matrix = viz.plot_feature_correlations(df)
    
    # Print highly correlated features
    print("\nHighly Correlated Feature Pairs (|r| > 0.7):")
    high_corr = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_val = corr_matrix.iloc[i, j]
            if abs(corr_val) > 0.7:
                high_corr.append((
                    corr_matrix.columns[i], 
                    corr_matrix.columns[j], 
                    corr_val
                ))
    
    for feat1, feat2, corr in sorted(high_corr, key=lambda x: abs(x[2]), reverse=True):
        print(f"  {feat1} <-> {feat2}: {corr:.3f}")

## 6. Identify Known Stellar Populations

In [ ]:
# Identify different stellar populations based on HR diagram position
if 'df' in locals():
    print("Stellar Population Analysis:")
    
    # Main sequence stars (rough criteria)
    main_sequence = df[
        (df['bp_rp'] > -0.5) & (df['bp_rp'] < 2.0) &
        (df['abs_g_mag'] > (4.83 + 5.5 * df['bp_rp'] - 2)) &
        (df['abs_g_mag'] < (4.83 + 5.5 * df['bp_rp'] + 2))
    ]
    print(f"  Main Sequence Stars: {len(main_sequence):,} ({len(main_sequence)/len(df)*100:.1f}%)")
    
    # White dwarfs (hot and faint)
    white_dwarfs = df[
        (df['bp_rp'] < 0.5) & (df['abs_g_mag'] > 10)
    ]
    print(f"  White Dwarf Candidates: {len(white_dwarfs):,} ({len(white_dwarfs)/len(df)*100:.1f}%)")
    
    # Red giants (cool and bright)
    red_giants = df[
        (df['bp_rp'] > 1.0) & (df['abs_g_mag'] < 4)
    ]
    print(f"  Red Giant Candidates: {len(red_giants):,} ({len(red_giants)/len(df)*100:.1f}%)")
    
    # High proper motion stars (potential nearby objects)
    if 'pm_total' in df.columns:
        high_pm = df[df['pm_total'] > 50]  # > 50 mas/yr
        print(f"  High Proper Motion Stars (>50 mas/yr): {len(high_pm):,} ({len(high_pm)/len(df)*100:.1f}%)")
    
    # High velocity stars (potential hypervelocity candidates)
    if 'v_tan_km_s' in df.columns:
        high_velocity = df[df['v_tan_km_s'] > 100]  # > 100 km/s
        print(f"  High Velocity Stars (>100 km/s): {len(high_velocity):,} ({len(high_velocity)/len(df)*100:.1f}%)")
        
        # Extreme velocity candidates
        extreme_velocity = df[df['v_tan_km_s'] > 300]  # > 300 km/s
        print(f"  Extreme Velocity Candidates (>300 km/s): {len(extreme_velocity):,}")
        
        if len(extreme_velocity) > 0:
            print("\n  Top 5 Highest Velocity Objects:")
            top_velocity = extreme_velocity.nlargest(5, 'v_tan_km_s')
            for idx, row in top_velocity.iterrows():
                print(f"    Source {row['source_id']}: {row['v_tan_km_s']:.1f} km/s")

## 7. Data Summary for Anomaly Detection

In [ ]:
# Summary for anomaly detection preparation
if 'df' in locals():
    print("=== Data Summary for Anomaly Detection ===")
    print(f"Total sources: {len(df):,}")
    print(f"Features available: {len(df.columns)}")
    
    # Key features for ML
    ml_features = [
        'parallax', 'pmra', 'pmdec', 'pm_total',
        'phot_g_mean_mag', 'abs_g_mag', 'bp_rp',
        'distance_pc', 'v_tan_km_s', 'reduced_pm', 'ruwe'
    ]
    
    available_ml_features = [f for f in ml_features if f in df.columns]
    print(f"ML-ready features: {len(available_ml_features)}")
    print(f"Features: {available_ml_features}")
    
    # Data quality for ML
    ml_df = df[available_ml_features]
    complete_cases = ml_df.dropna()
    print(f"\nComplete cases for ML: {len(complete_cases):,} ({len(complete_cases)/len(df)*100:.1f}%)")
    
    print("\nReady for preprocessing and anomaly detection!")
    print("Next steps:")
    print("1. Run ../src/preprocessing.py to prepare training data")
    print("2. Run ../src/models.py to train anomaly detection models")
    print("3. Analyze results and create visualizations")

## Conclusions

This exploratory analysis provides a foundation for anomaly detection by:

1. **Data Quality**: Confirming the dataset is clean and suitable for ML
2. **Stellar Populations**: Identifying known populations that should be "normal"
3. **Feature Understanding**: Understanding correlations and distributions
4. **Baseline Expectations**: Setting expectations for what constitutes an anomaly

The next phase will involve training unsupervised ML models to identify objects that deviate significantly from these known populations.